In [2]:
import os
import numpy as np
from ultralytics import YOLO
import torch

In [3]:
def find_closest_index(d, y):
    closest_index = 0
    closest_distance = abs(d[0] - y)
    
    for i in range(1, len(d)):
        distance = abs(d[i] - y)
        if distance < closest_distance:
            closest_index = i
            closest_distance = distance
    
    return closest_index

In [4]:
# Finding y distances
a = [0.35, 0.417083, 0.337917, 0.423333, 0.330000]
b = [0.425, 0.494167, 0.418333, 0.501667, 0.410833]
c = [0.508333, 0.579583, 0.507917, 0.588333, 0.500000]
d = [0.584167, 0.657500, 0.587500, 0.667917, 0.580000]
bot = [0.282917, 0.349583, 0.265000, 0.354583, 0.256667]
top = [0.651250, 0.728750, 0.657917, 0.738750, 0.650417]
ay = []
by = []
cy = []
dy = []
for i in range(5):
  ay.append(a[i]-bot[i])
  by.append(b[i]-bot[i])
  cy.append(c[i]-bot[i])
  dy.append(d[i]-bot[i])
print(ay)
print(np.mean(ay))
print(by)
print(np.mean(by))
print(cy)
print(np.mean(cy))
print(dy)
print(np.mean(dy))
y_lines = [ np.mean(dy), np.mean(cy), np.mean(by) ,np.mean(ay)]
print(y_lines)

[0.067083, 0.0675, 0.07291700000000001, 0.06875000000000003, 0.07333300000000004]
0.06991660000000002
[0.14208300000000001, 0.14458400000000005, 0.153333, 0.147084, 0.15416600000000003]
0.14825000000000002
[0.22541600000000006, 0.22999999999999998, 0.24291699999999994, 0.23375, 0.24333300000000002]
0.2350832
[0.30125, 0.307917, 0.3225, 0.313334, 0.323333]
0.3136668
[0.3136668, 0.2350832, 0.14825000000000002, 0.06991660000000002]


In [15]:
path = os.path.join('Datasets','Expandidas')
output_path = os.path.join('Datasets','Expandidas','detecting')
train_files = os.listdir(os.path.join(path, 'training', 'labels'))
val_files = os.listdir(os.path.join(path, 'validation', 'labels'))
file_names = {'training': train_files, 'validation': val_files}

for folder in ['training', 'validation']:
  file_path = os.path.join(path, folder, 'labels')
  for file_name in file_names[folder]:
    if file_name != 'classes.txt':
      with open(os.path.join(file_path,file_name), 'r') as file:
        # Defining variables
        marks = []
        columns = []
        board = None
        xy_marks = []
        # Reading data
        lines = file.readlines()
        for line in lines:
          if line[0] == '0':
            marks.append(line)
          elif line[0] == '1':
            columns.append(line)
          elif line[0] == '2':
            board = line
          else:
            raise Exception('Class not defined') 
        # Finding y bottom
        assert board, 'Board class not found'
        assert len(columns) == 20, 'The number of columns is not equal to 20'
        board = board.split(' ')
        height = float(board[4].replace('\\','').replace('n',''))
        y_center = float(board[2])
        y_bottom = y_center + (height/2)
        # Finding x columns
        x_columns = []
        for column in columns: 
          x_columns.append(float(column.split(' ')[1]))
        # Finding positions
        for mark in marks:
          mark = mark.split(' ')
          x = find_closest_index(x_columns, float(mark[1]))
          y = find_closest_index(y_lines, (y_bottom-float(mark[2])))
          xy_marks.append({'x':x, 'y':y})
        output_file_path = os.path.join(output_path, file_name)
        with open(output_file_path, 'w') as f:
          for mark in xy_marks:
              f.write(f"{mark['x']} {mark['y']}\n")

        


In [10]:
model = YOLO(r'Runs/detect/train11/weights/best.pt')

In [12]:
# Running Inference
path = os.path.join('Datasets','Expandidas')
output_path = os.path.join('Datasets','Expandidas','inference')
train_files = os.listdir(os.path.join(path, 'training', 'images'))
val_files = os.listdir(os.path.join(path, 'validation', 'images'))
file_names = {'training': train_files, 'validation': val_files}

for folder in ['training', 'validation']:
  for file_name in file_names[folder]:
    results = model(f'Datasets/Expandidas/{folder}/images/{file_name}')
    for result in results:
      # Defining variables
      marks = []
      columns = []
      board = None
      xy_marks = []
      # Reading data
      for i, cls  in enumerate(result.boxes.cls):
        if cls == 0.0:
          marks.append(result.boxes.xywhn[i])
        elif cls == 1.0:
          columns.append(result.boxes.xywhn[i])
        elif cls == 2.0:
          board = result.boxes.xywhn[i]
        else:
          raise Exception('Class not defined') 
      # Finding y bottom
      assert len(board.size()) != 0, 'Board class not found'
      assert len(columns) == 20, 'The number of columns is not equal to 20'
      height = board[3]
      y_center = board[1]
      y_bottom = y_center + (height/2)
      # Finding x columns
      x_columns = []
      for column in columns: 
        x_columns.append(column[0])
      x_columns.sort()
      # Finding positions
      for mark in marks:
        x = find_closest_index(x_columns, mark[0])
        y = find_closest_index(y_lines, (y_bottom-mark[1]))
        xy_marks.append({'x':x, 'y':y})
      output_file_path = os.path.join(output_path, file_name.replace('.jpeg','.txt').replace('.jpg','.txt'))
      with open(output_file_path, 'w') as f:
        for mark in xy_marks:
          f.write(f"{mark['x']} {mark['y']}\n")


image 1/1 c:\Users\Tiago\Documents\FITec\ComputerVision\SAIP\Datasets\Expandidas\training\images\1_1.jpeg: 1216x1600 6 Os, 20 Colunas, 1 Canto, 231.5ms
Speed: 15.0ms preprocess, 231.5ms inference, 1.0ms postprocess per image at shape (1, 3, 1216, 1600)

image 1/1 c:\Users\Tiago\Documents\FITec\ComputerVision\SAIP\Datasets\Expandidas\training\images\1_10.jpg: 1216x1600 4 Os, 20 Colunas, 1 Canto, 30.2ms
Speed: 5.0ms preprocess, 30.2ms inference, 1.0ms postprocess per image at shape (1, 3, 1216, 1600)

image 1/1 c:\Users\Tiago\Documents\FITec\ComputerVision\SAIP\Datasets\Expandidas\training\images\1_3.jpg: 1216x1600 5 Os, 20 Colunas, 1 Canto, 30.0ms
Speed: 5.0ms preprocess, 30.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1216, 1600)

image 1/1 c:\Users\Tiago\Documents\FITec\ComputerVision\SAIP\Datasets\Expandidas\training\images\1_5.jpg: 1216x1600 3 Os, 20 Colunas, 1 Canto, 30.1ms
Speed: 5.0ms preprocess, 30.1ms inference, 1.0ms postprocess per image at shape (1, 3, 1216, 1

In [16]:
def compare_files(file_path1, file_path2, file_name):
    errors = []
    with open(file_path1 + '/' + file_name, 'r') as file1, open(file_path2 + '/' + file_name, 'r') as file2:
        lines1 = file1.readlines()
        lines2 = file2.readlines()
        lines1 = [line.strip() for line in lines1]
        lines2 = [line.strip() for line in lines2]
        if set(lines1) != set(lines2):
            errors.append(file_name)
    return errors

inference_path = os.path.join('Datasets','Expandidas','inference')
detecting_path = os.path.join('Datasets','Expandidas','detecting')
for file in os.listdir(os.path.join('Datasets','Expandidas','inference')):
  print(compare_files(inference_path,detecting_path,file))

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
